In [ ]:
import datetime
import os
import shutil
from pathlib import Path

import numpy as np
from joblib import Parallel, delayed
from simple_slurm import Slurm
from tqdm import tqdm

In [ ]:
# Refactor + Rerun
output_path = Path(
    "/Morphodynamics_of_human_early_brain_organoid_development/tracking/lumen_tracking"
)
morpho_path_all = Path(output_path, f"morphometrics")
track_path_all = Path(output_path, f"tracks")

In [ ]:
# Create slurm jobs for lumen tracking
positions = np.arange(1, 17)
for position in positions:
    mask_dir = f"/20220611_200804_ActGFP_LamRFP_TubRFFP_WTC_HB4_D4_pos1-4Mat_9-12_agar_processed/Position_{position}_Settings_1_Processed/lumen_masks_v19_05_2023/"
    h5_file = f"/20220611_200804_ActGFP_LamRFP_TubRFFP_WTC_HB4_D4_pos1-4Mat_9-12_agar_processed/Position_{str(position)}_Settings_1_Processed/denoised_processed.h5"
    out_path = Path(output_path, f"Position_{str(position)}_Settings_1_Processed.zarr")
    morpho_path = Path(morpho_path_all, f"Position_{position}_Settings_1_Processed")
    track_path = Path(track_path_all, f"Position_{position}_Settings_1_Processed")
    registered_path = Path(
        output_path,
        f"Position_{str(position)}_Settings_1_Processed.zarr".replace(
            ".zarr", "_registered.zarr"
        ),
    )
    Path(track_path).mkdir(parents=True, exist_ok=True)
    Path(morpho_path).mkdir(parents=True, exist_ok=True)
    label = "cell_segmentation"
    channel = "GFP"
    tissue_label = "tissue_mask"
    smootheness = 5

    slurm = Slurm(
        cpus_per_task=12,
        mem_per_cpu=10000,
        job_name=f"h5_ome_zarr_{position}",
        output=f"logs/job_%j.out",
        account="es_treutlein",
        time=datetime.timedelta(days=0, hours=4, minutes=0, seconds=0),
    )

    slurm.add_cmd("source activate light_insight")
    job_name_converter = slurm.sbatch(
        f"python steps/h5_omezarr_conversion.py --h5_file {h5_file} --out_path {out_path}",
        shell="/bin/bash",
    )

    slurm = Slurm(
        cpus_per_task=8,
        mem_per_cpu=3000,
        job_name=f"add_tissue_segmentation_{position}",
        dependency=dict(afterok=job_name_converter),
        output=f"logs/job_%j.out",
        error=f"error/job_%j.out",
        account="es_treutlein",
        time=datetime.timedelta(days=0, hours=4, minutes=0, seconds=0),
    )

    slurm.add_cmd("source activate light_insight")
    job_name_tissue_segment = slurm.sbatch(
        f"python steps/add_tissue_segmentation.py --mask_dir {mask_dir} --input_path {out_path}",
        shell="/bin/bash",
    )

    # Run registration
    slurm = Slurm(
        cpus_per_task=8,
        mem_per_cpu=5000,
        dependency=dict(afterok=job_name_tissue_segment),
        job_name=f"registration_{position}",
        output=f"logs/job_%j.out",
        error=f"error/job_%j.out",
        account="es_treutlein",
        time=datetime.timedelta(days=0, hours=4, minutes=0, seconds=0),
    )

    slurm.add_cmd("source activate light_insight")
    job_name_registration = slurm.sbatch(
        f"python steps/register_movie.py --input_path {out_path} --channel {channel}",
        shell="/bin/bash",
    )

    # Run tissue smoothing
    slurm = Slurm(
        cpus_per_task=8,
        mem_per_cpu=16000,
        dependency=dict(afterok=job_name_registration),
        job_name=f"Smoothing_{position}",
        output=f"logs/job_%j.out",
        error=f"error/job_%j.out",
        account="es_treutlein",
        time=datetime.timedelta(days=0, hours=4, minutes=0, seconds=0),
    )

    slurm.add_cmd("source activate light_insight")
    job_name_filter = slurm.sbatch(
        f"python steps/temporal_filter.py --input_path {registered_path} --kernel_size {smootheness} --label {tissue_label} --channel {channel}",
        shell="/bin/bash",
    )

    label_name_smooth = f"lumen_masks_smooth_{smootheness}_processed"

    # Run tracking tissue
    slurm = Slurm(
        cpus_per_task=8,
        mem_per_cpu=8000,
        dependency=dict(afterok=job_name_filter),
        job_name=f"tracking_{position}",
        output=f"logs/job_%j.out",
        error=f"error/job_%j.out",
        account="es_treutlein",
        time=datetime.timedelta(days=0, hours=4, minutes=0, seconds=0),
    )

    slurm.add_cmd("source activate light_insight")
    slurm.add_cmd("module load gurobi/10.0.3")
    job_name_tracking = slurm.sbatch(
        f"python steps/track_movie.py --input_path {registered_path} --output_path {track_path}/ --label {label_name_smooth} --channel {channel}",
        shell="/bin/bash",
    )

    channel = "GFP"
    tissue_label = "tissue_mask"
    smootheness = 5
    morpho_label_name = "lumen_masks_smooth_5_processed_detection"
    n_jobs = 4
    zarr_level = 1

    slurm = Slurm(
        array=range(0, 6),
        cpus_per_task=4,
        mem_per_cpu=800,
        dependency=dict(afterok=job_name_tracking),
        job_name="extract_morphometrics",
        output=f"logs/job_%j.out",
        error=f"error/job_%j.out",
        account="es_treutlein",
        time=datetime.timedelta(days=0, hours=12, minutes=0, seconds=0),
    )

    slurm.add_cmd("source activate light_insight")
    job_name_morpho = slurm.sbatch(
        f"python steps/measure_morphometrics.py --input_path {registered_path}"
        + f" --morpho_path {morpho_path}/"
        + f" --array_section {Slurm.SLURM_ARRAY_TASK_ID}"
        + f" --label_name {morpho_label_name}"
        + f" --n_jobs {n_jobs}"
        + f" --zarr_level {zarr_level}",
        shell="/bin/bash",
    )